In [1]:
import torch.nn as nn
import torch.optim as optim
from  torchvision import models

In [2]:
conv = nn.Conv2d(1,1, kernel_size=1)


In [3]:
params = list(conv.parameters())

In [4]:
named_params = list(conv.named_parameters())

In [5]:
params

[Parameter containing:
 tensor([[[[-0.3925]]]], requires_grad=True), Parameter containing:
 tensor([0.9319], requires_grad=True)]

In [6]:
named_params

[('weight', Parameter containing:
  tensor([[[[-0.3925]]]], requires_grad=True)), ('bias', Parameter containing:
  tensor([0.9319], requires_grad=True))]

In [2]:
class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()
        self.conv1 = nn.Conv2d(1,1, kernel_size=3)
        self.conv2 = nn.Conv2d(1,1, kernel_size=3)
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(1,1, kernel_size=3),
            nn.Conv2d(1,1, kernel_size=3),
            nn.MaxPool2d(2,2)
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(1,1, kernel_size=3),
            nn.Conv2d(1,1, kernel_size=3),
            nn.MaxPool2d(2,2)
        )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        return x
    
model = TestNet()

In [8]:
mparam = list(model.parameters())
nparam = list(model.named_parameters())

In [9]:
mparam

[Parameter containing:
 tensor([[[[ 0.2056,  0.2884,  0.0164],
           [-0.1758,  0.0430,  0.1899],
           [-0.1654,  0.0868, -0.2053]]]], requires_grad=True),
 Parameter containing:
 tensor([-0.1598], requires_grad=True),
 Parameter containing:
 tensor([[[[ 0.2011, -0.2741,  0.1188],
           [-0.1493, -0.1230, -0.2897],
           [-0.0751,  0.1814,  0.1577]]]], requires_grad=True),
 Parameter containing:
 tensor([0.0479], requires_grad=True),
 Parameter containing:
 tensor([[[[-0.0548, -0.0474, -0.0260],
           [-0.2184,  0.0244,  0.1152],
           [ 0.0044, -0.1206, -0.3273]]]], requires_grad=True),
 Parameter containing:
 tensor([-0.0608], requires_grad=True),
 Parameter containing:
 tensor([[[[-0.0322,  0.0542,  0.1376],
           [ 0.2496, -0.1909, -0.0437],
           [-0.2114, -0.2389,  0.1166]]]], requires_grad=True),
 Parameter containing:
 tensor([-0.1664], requires_grad=True),
 Parameter containing:
 tensor([[[[ 0.0519,  0.0189,  0.1214],
           [ 0.248

In [10]:
key = []
for p in nparam:
    print(p[0]) 

conv1.weight
conv1.bias
conv2.weight
conv2.bias
conv_block1.0.weight
conv_block1.0.bias
conv_block1.1.weight
conv_block1.1.bias
conv_block2.0.weight
conv_block2.0.bias
conv_block2.1.weight
conv_block2.1.bias


In [11]:
optim.SGD([
    {'params':model.conv1.parameters(), 'lr':0.001},
    {'params':model.conv2.parameters(), 'lr':0.001},
    
], lr=0.001)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    lr: 0.001
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [3]:
def get_name(name_split):
    if len(name_split)<=2:
        key = name_split[-2]
    else:
        key = '.'.join(name_split[:-1]) 
    return key

def print_named_params(model):
    named_params = model.named_parameters()
    layer_group = collections.OrderedDict()
    pair_group = {}
    param_group = []
    last_name = ""
    result = None
    for idx, (name, param) in enumerate(named_params):
        name_split = name.split(".")
        layer_name, name_type = name_split[-2], name_split[-1]          
        usable_name = f'{layer_name}.{name_type}'
        key = get_name(name_split)
#         print(name)
        if idx==0:
            param_group.append((usable_name, ''))
            layer_group[key] = param_group
            last_name = layer_name
        else:
            if last_name == layer_name:
                pair_group[usable_name] = ''
                param_group.append((usable_name,''))
                layer_group[key] = param_group
            else:
                param_group = []
                param_group.append((usable_name,''))
                layer_group[key] = param_group
        last_name = layer_name
    result = layer_group
    return result

def create_param_group(model):
    named_params = model.named_parameters()
    layer_group = collections.OrderedDict()
    pair_group = {}
    param_group = []
    last_name = ""
    result = None
    for idx, (name, param) in enumerate(named_params):
        name_split = name.split(".")
        layer_name, name_type = name_split[-2], name_split[-1]          
        usable_name = f'{layer_name}.{name_type}'
        key = get_name(name_split)
#         print(name)
        if idx==0:
#             param_group.append((usable_name, 'param'))
            param_group.append(param)
            layer_group[key] = param_group
            last_name = layer_name
        else:
            if last_name == layer_name:
#                 param_group.append((usable_name,'param'))
                param_group.append(param)
                layer_group[key] = param_group
            else:
                param_group = []
#                 param_group.append((usable_name,'param'))
                param_group.append(param)
                layer_group[key] = param_group
        last_name = layer_name
    result = layer_group
    return result

In [4]:
test_model = models.resnet18()
param_group = create_param_group(test_model)
for k,v in param_group.items():
    print(k)

NameError: name 'collections' is not defined

In [199]:
def test_param_group(test_model):
    param_group = create_param_group(test_model)
    parameters = list(test_model.parameters())
    named_parameters = list(test_model.named_parameters())
    
    idx = 0
    for key, values in param_group.items():
        for value in values:
            if value is parameters[idx]:
                print(named_parameters[idx][0], key)
            idx = idx+1
    print(idx == len(named_parameters))
test_param_group(test_model)

conv1.weight conv1
bn1.weight bn1
bn1.bias bn1
layer1.0.conv1.weight layer1.0.conv1
layer1.0.bn1.weight layer1.0.bn1
layer1.0.bn1.bias layer1.0.bn1
layer1.0.conv2.weight layer1.0.conv2
layer1.0.bn2.weight layer1.0.bn2
layer1.0.bn2.bias layer1.0.bn2
layer1.1.conv1.weight layer1.1.conv1
layer1.1.bn1.weight layer1.1.bn1
layer1.1.bn1.bias layer1.1.bn1
layer1.1.conv2.weight layer1.1.conv2
layer1.1.bn2.weight layer1.1.bn2
layer1.1.bn2.bias layer1.1.bn2
layer2.0.conv1.weight layer2.0.conv1
layer2.0.bn1.weight layer2.0.bn1
layer2.0.bn1.bias layer2.0.bn1
layer2.0.conv2.weight layer2.0.conv2
layer2.0.bn2.weight layer2.0.bn2
layer2.0.bn2.bias layer2.0.bn2
layer2.0.downsample.0.weight layer2.0.downsample.0
layer2.0.downsample.1.weight layer2.0.downsample.1
layer2.0.downsample.1.bias layer2.0.downsample.1
layer2.1.conv1.weight layer2.1.conv1
layer2.1.bn1.weight layer2.1.bn1
layer2.1.bn1.bias layer2.1.bn1
layer2.1.conv2.weight layer2.1.conv2
layer2.1.bn2.weight layer2.1.bn2
layer2.1.bn2.bias layer2.

In [185]:
param_group['layer1.0.conv1'][0] is list(test_model.parameters())[3]

True

In [300]:
# for c in list(test_model.children()):
#     print(c)
#     try:
#         cc = c.children()
#         print(list(cc))
#     except:
#         pass
#     print()
    
def flatten_model(model):
    module = []
    for mod in list(model.children()):
        if has_child(mod):
            
        sub = list(mod.children())
        if len(sub)>0:
#             flatten_model(mod)
            print(sub)
        else:
            print('no children')
            module.append(mod)
            
    return module
                
       
        
                

def has_child(mod):
    try:
        sub = list(mod.children())
        if len(sub)>0:
            return True
        else:
            return False
    except:
        return False


mod = list(test_model.children())
has_child(mod[0])
    
# flatten_model(test_model)        

False

In [303]:
list(test_model.modules())

[ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace)


In [5]:
class ModelParser:
    @staticmethod
    def get_name(name_split):
        if len(name_split)<=2:
            key = name_split[-2]
        else:
            key = '.'.join(name_split[:-1])
        return key

    @staticmethod
    def get_params_group(model):
        layer_group = collections.OrderedDict()
        param_group, last_name = [], ""
        for idx, (name, param) in enumerate(model.named_parameters()):
            name_split = name.split(".")
            layer_name, name_type = name_split[-2], name_split[-1]
            key = ModelParser.get_name(name_split)
            if idx == 0:
                param_group.append(param)
                layer_group[key] = param_group
                last_name = layer_name
            else:
                if last_name == layer_name:
                    param_group.append(param)
                    layer_group[key] = param_group
                else:
                    param_group = []
                    param_group.append(param)
                    layer_group[key] = param_group
            last_name = layer_name
        return layer_group

In [6]:
ModelParser.get_params_group(test_model).keys()

NameError: name 'test_model' is not defined

In [78]:
from typing import *
import collections
class OptimizerWrapper(object):
    def __init__(self, model: nn.Module, optimizer:optim.Optimizer = None):
        self.model = model
        self.optimizer: optim.Optimizer = optimizer
        
        self.lr_decay = 0
        self.momentum = 0
        self.beta1, self.beta2 = 0.9, 0.999
        self.betas = [self.beta1, self.beta2]
        self.weight_decay = 0
        self.eps = 1e-08
        self.alpha = 0.99
        self.dampening = 0
        self.nesterov = False
    
    def create(self, lr: Union[float, list], name='sgd', **kwargs):
        model_params = self._model_parameters(lr)
        if self.optimizer:
            return self.optimizer(model_params)
        else:
            self.optimizer = optim.Adam
            return self.optimizer(model_params)
        
        
    def _model_parameters(self, lr:Union[float, list], **kwargs):
        parameters = self._model_param_group()
        if type(lr) == list:
            build = []
            lr_len = len(lr)
            param_len = len(parameters)
            lr_every = None
            if param_len < lr_len:
                raise ValueError('lr list is more bigger than model parameter, try to remove item on list!')
            else:
                 lr_every = param_len // len(lr)
            
            lr_idx = 0
            for idx, param in enumerate(parameters):   
                if idx % lr_every == 0 and idx!=0:
                    lr_idx+=1
                if len(lr)<=lr_idx:
                    lr_idx-=1
#                 print(idx, lr[lr_idx])
                par = {'params': param, 'lr':lr[lr_idx]}
                build.append(par)
            return build
        else:
            return parameters
            
    
    def _lr_every(self,param, lr):
        param_len = len(param)
        lr_every = param_len // len(lr)
        lr_list = []
        for idx in range(param_len):
            if  idx % lr_every == 0 and id!=0:
                lr_list.append(idx)
        return lr_list

    
    def _model_param_group_odict(self):
        return ModelParser.get_params_group(self.model)
    
    def _model_param_group(self):
        parameters = []
        for idx, (name, param) in enumerate(self._model_param_group_odict().items()):
            parameters.append(param)
        return parameters
    
    

test_model = models.alexnet()
opt = optim.Adam        
optwr = OptimizerWrapper(test_model)
optwr.create(lr=[0.1], betas=[0.5,0.999])

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0

Parameter Group 2
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0

Parameter Group 3
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0

Parameter Group 4
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0

Parameter Group 5
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0

Parameter Group 6
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0

Parameter Group 7
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0
)

In [318]:
import torch.optim as optim
a = optim.SGD

In [319]:
sgd(test_model.parameters(), lr=0.1)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [18]:

# [idx for idx in range(param_len)]

def get_lr_every(param, lr):
    param_len = len(param)
    lr_len = len(lr)
    lr_every = param_len // lr_len
    lr_list = []
    for idx in range(param_len):
        if  idx % lr_every == 0 and idx!=0:
            lr_list.append(idx)
    return lr_list[:-1]
    


In [363]:
every = 8
c = 0 
for i in range(1,10):
    if i % every == 0:
        print(i)

2
4
6
8


In [39]:
8//4

2

In [90]:
optim.Adam == optim.Adamax

False

In [91]:
a = [1,2,3,4,5,100]
a[:-1]

[1, 2, 3, 4, 5]